In [1]:
import aug_util as aug
import wv_util as wv
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import numpy as np
import csv
import tqdm
%matplotlib inline


In [4]:
#Loading our labels
coords1, chips1, classes1 = wv.get_labels('/data/zjc4//xView_train.geojson')

100%|██████████| 601937/601937 [00:03<00:00, 153159.13it/s]


In [5]:
# Grouped classes [1,2,3,4,5]
grouped_classes = [[11,12],[13],[17,18,20,21],\
       [19,23,24,25,28,29,60,61,65,26],[41,42,50,40,44,45,47,49]]
def transpose_class_counts(chips,classes):
    chip_names = np.unique(chips) 
    ### Uncomment this to run over entire train dataset ###
    # chip_names = np.unique(chips) 
    results = np.zeros((len(chip_names),5))
    chip_strs = []
    for c_idx, c in tqdm.tqdm(enumerate(chip_names)):
        chip_strs.append(c)
        classes_chip = classes[chips==c]
        idx_filter = np.isin(classes_chip,grouped_classes[0])
        # initialize to all false
        for i,gc in (enumerate(grouped_classes)):
            is_in_idxs = np.isin(classes_chip,gc)
            classes_chip[is_in_idxs] = i
            idx_filter = np.logical_or(idx_filter,is_in_idxs)
        classes_chip = classes_chip[idx_filter]
        labels, counts = np.unique(classes_chip,return_counts=True)
        for label_idx,label in enumerate(labels):
            results[int(c_idx),int(label)] = counts[label_idx]
            pass
    chip_strs_col = np.array(chip_strs).reshape(-1,1)
    return (np.hstack((chip_strs_col,results)))

# make the table where filename is rowwise, columns are class
results = transpose_class_counts(chips1,classes1)

847it [00:27, 30.30it/s]


In [7]:
df

,fname,Small Aircraft,Large Aircraft,Small Vehicle,Bus/Truck,Boat
0,10.tif,0.0,0.0,2.0,1.0,0.0
1,100.tif,0.0,0.0,62.0,7.0,0.0
2,102.tif,0.0,0.0,346.0,24.0,0.0
3,1036.tif,0.0,0.0,31.0,36.0,69.0
4,1037.tif,0.0,0.0,276.0,146.0,0.0
...,...,...,...,...,...,...
842,97.tif,0.0,0.0,934.0,61.0,0.0
843,983.tif,1.0,2.0,0.0,1.0,0.0
844,986.tif,0.0,0.0,7.0,3.0,0.0
845,99.tif,0.0,0.0,851.0,81.0,0.0


In [6]:
# the train test split code starts here

In [7]:
no_tif_col = results[:, 1:]

In [8]:
no_tif_col

array([['0.0', '0.0', '2.0', '1.0', '0.0'],
       ['0.0', '0.0', '62.0', '7.0', '0.0'],
       ['0.0', '0.0', '346.0', '24.0', '0.0'],
       ...,
       ['0.0', '0.0', '7.0', '3.0', '0.0'],
       ['0.0', '0.0', '851.0', '81.0', '0.0'],
       ['0.0', '0.0', '0.0', '0.0', '0.0']], dtype='<U32')

In [10]:
from sklearn.model_selection import train_test_split

def indToTifName(data, inds):
    res = []
    for ind in inds:
        res.append(data[ind][0])
    return res

def showDistribution(data, selected_indexes):
    res = []
    total = 0
    class_num = len(data[0])
    for i in range(class_num):
        for index in selected_indexes:
            total += float(data[index][i])
    for i in range(class_num):
        total_of_this_class = 0
        for index in selected_indexes:
            total_of_this_class += float(data[index][i])
        res.append(float(total_of_this_class)/total)
    return res

def checkThreshold(distr1, distr2, thres):
    if (len(distr1) != len(distr2)):
        print("columns' numbers don't fit.")
        return -1
    for i in range(len(distr1)):
        diff = abs(distr1[i] - distr2[i])
        if diff > thres:
            return False
    return True

def findBalance(data, train_percent, thres):
    tifs = len(data)
    class_num = len(data[0])
    for i in range(1000000):
        tr_set, te_set = train_test_split(np.array(list(range(len(data)))), test_size=1-train_percent)
        tr_d = showDistribution(data, tr_set)
        te_d = showDistribution(data, te_set)
        check = checkThreshold(tr_d, te_d, thres)
        if (check == -1):
            return -1
        elif (check == True):
            return tr_set, te_set
    return [], []

In [11]:
#input data (here "no_tif_col") has to be complete columns of classes of counts, no other columns like "tif_names"
#set train test split percentage (0.8) and threshold here (0.02)
train_ind, test_ind = findBalance(no_tif_col, 0.8, 0.02)

In [12]:
train_ind

array([392, 420, 678, 305, 761, 715, 608, 515, 778, 455, 226, 757, 447,
       664, 456, 174, 334,  94,  86, 396, 699, 414,  42, 308, 160, 668,
       816, 547, 369, 429,  62, 292, 342, 346, 835, 196, 670, 118, 467,
       552, 362, 713, 609, 635, 829, 768, 485, 105, 646, 257, 313, 566,
       751, 252,  89,   5, 260, 575, 568, 769,  29, 494, 322, 320, 755,
       361,  96, 616, 285,  23, 137, 465, 517,  40, 130, 281, 564, 626,
       679, 686, 302, 223, 215, 114, 676, 221, 694, 169, 837, 544, 359,
       821, 773, 711, 181, 605, 551, 208,  53, 156,  68,   6, 653,  82,
       451, 425, 610, 656, 338, 845, 613, 473, 407, 178, 377, 123, 692,
       371, 620,  77,  37, 483, 530,  25, 171, 493, 841, 807, 225, 705,
       634, 489, 418, 637, 432, 743, 640, 375, 124, 412, 211, 300, 182,
       337, 712, 203, 509, 201, 180, 741, 280,   0, 312, 524, 444, 390,
       660,  24,  90, 759, 310, 404, 594, 710, 683, 738, 698, 822, 601,
       470, 657, 339,  59, 790, 380, 401, 632, 636, 164, 796, 37

In [13]:
test_ind

array([628, 565, 800, 235, 448, 307, 117,  12,  85, 824, 132, 188, 619,
        57, 469, 245, 446,  41, 185,  91, 529, 333, 439, 492, 406, 554,
       667, 301, 135, 728, 150, 671, 289, 780,  47, 233, 266, 726, 131,
        78, 288, 259, 721,  83, 748, 112, 127, 758, 283, 356, 581,   7,
       352, 633, 478, 525, 100, 540, 391, 502, 611, 146, 724, 454, 328,
       722,  80, 168, 435, 617, 839, 424, 621, 779, 466, 462, 393, 573,
       271, 487, 799, 148, 372, 499,   2, 306, 602, 411, 152,  18, 561,
       163, 416, 507, 589,   4, 846, 309, 430, 363, 385,  71,  56,  63,
        49, 270, 374, 583, 491, 586,  58, 317, 782, 651, 238, 251, 718,
       688, 793, 438, 395, 553, 580, 639, 199, 236, 350, 495, 827,  14,
        65, 772, 590, 735, 531, 791, 550, 119, 458,  27, 325, 136, 781,
       716, 519, 815, 145, 690, 643, 243, 216,  50, 503, 497, 443, 433,
       819, 729,  95, 486,  61, 584, 125,  30, 484, 644, 107, 834, 477,
       193])

In [14]:
# distributions
showDistribution(no_tif_col, train_ind)

[0.0015955025833137438,
 0.002815887004406388,
 0.8517783101677028,
 0.12484932753817453,
 0.018960972706402516]

In [15]:
showDistribution(no_tif_col, test_ind)

[0.002123765453415291,
 0.0026762897990192692,
 0.8470543545824988,
 0.1320015194419504,
 0.016144070723116237]

In [16]:
#result tifs, need to use dataset with tif names as the input
train_tifs = indToTifName(results, train_ind)
test_tifs = indToTifName(results, test_ind)

In [24]:
import pickle
with open("train_tifs.pkl","wb") as f:
    pickle.dump(train_tifs,f)
    pass
with open("test_tifs.pkl","wb") as f:
    pickle.dump(test_tifs,f)
    
    